In [1]:
from pymongo import MongoClient

# Connect to your MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["myDatabase"]
collection = db["myCollection"]

# Fetch your scraped documents (assume stored under 'content' field)
documents = [doc['page_text'] for doc in collection.find()]


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Chunk and Preprocess the data
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs = splitter.create_documents(documents)


In [3]:
docs

[Document(metadata={}, page_content='Buffer\n¶\nclass\ntorch.nn.parameter.\nBuffer\n(\ndata\n=\nNone\n,\n*\n,\npersistent\n=\nTrue\n)\n[source]\n[source]\n¶\nA kind of Tensor that should not be considered a model\nparameter. For example, BatchNorm’s\nrunning_mean\nis not a parameter, but is part of the module’s state.\nBuffers are\nTensor\nsubclasses, that have a\nvery special property when used with\nModule\ns – when they’re\nassigned as Module attributes they are automatically added to the list of\nits buffers, and will appear e.g. in\nbuffers()\niterator.'),
 Document(metadata={}, page_content='buffers()\niterator.\nAssigning a Tensor doesn’t have such effect. One can still assign a Tensor as explicitly by using\nthe\nregister_buffer()\nfunction.\nParameters\ndata\n(\nTensor\n) – buffer tensor.\npersistent\n(\nbool\n,\noptional\n) – whether the buffer is part of the module’s\nstate_dict\n. Default:\nTrue\nNext\nPrevious\n© Copyright 2024, PyTorch Contributors.\nBuilt with\nSphinx\nu

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

# Generate embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


C:\Users\phani\AppData\Local\Temp\ipykernel_37392\4070198614.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [5]:
from langchain.vectorstores import FAISS

# Checkpoint for Vector Store
vectorstore = FAISS.from_documents(docs, embedding_model)
vectorstore.save_local("faiss_index")


c:\Users\phani\miniconda3\envs\rag_env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
# Load saved Vector Store
vectorstore = FAISS.load_local("faiss_index", embedding_model)